In [47]:
import pso
import particle
import os
from utils import load_cloud_dataset, load_breast_cancer, load_seg_data, load_digits_dataset, load_satelite_dataset, load_synthetic_dataset
import importlib
import numpy as np


In [64]:
importlib.reload(particle)
importlib.reload(pso)

<module 'pso' from '/home/skholkin/projects/PSO_GMM/pso.py'>

In [65]:
config = pso.PSOConfig.from_json(os.path.join('configs', 'default_params.json'))

data = load_breast_cancer()
# synth_dataset_name = 'Synthetic_dim_50_n_samples_1000_n_comp_15_c_separation_1.0.data.npy'
# data = load_synthetic_dataset(synth_dataset_name)
n_runs = 5
res = []
for i in range(n_runs):
    pso_algo = pso.PSOEigen(data, config)
    res.append(pso_algo.run())

for key in res[0].keys():
    value = np.mean([item[key] for item in res])
    std = np.std([item[key] for item in res])
    print(key, ': ', value, ' +- ', std)

EIGENVALUES
1000000.0000015451 222804.65529361478 10.500904685784246
503171.2497415487 46802.96368229134 21.63894370758232
1000000.0000018503 244715.24548013258 4.805923404624614
393870.03645539936 30763.40582290684 9.478163306699987
813129.6763691375 76979.28471067852 22.19827612667513
1000000.0000119535 533451.1470847252 3.52845211513864
462406.4006508966 41812.72360105452 10.64992529580464
714619.1394026444 66597.1057212126 19.831140356077107
1000000.0000086067 533488.5874825991 4.94679984935979
526025.9687716257 40068.82710843089 9.349989085395267
Max egival for random scattering:  4006.8827108430887
em :  73.59465820908106  +-  0.0
pso :  74.58922107423383  +-  0.0
ref_em :  74.63036707963411  +-  0.0


In [59]:
res_dict = pso_algo.run()

EIGENVALUES
1000000.000005306 433886.38766594057 5.434967403384544
440383.22067199345 30190.48854188338 10.423094140519735
698172.8895151264 59298.76011480407 15.330658336245865
1000000.0000066959 700069.8482806097 10.597123033321955
1000000.0000078125 566835.4528054115 4.024749282315477
1000000.0000085386 466951.1656378706 3.518141313644792
273815.855367959 21353.803394510986 5.472195830869321
1000000.0000008665 343112.9839048189 10.475700066082544
351663.7229727843 27062.555771612486 15.353888837752136
700844.7835859675 71654.01215549884 22.07853415738265
Max egival for random scattering:  7165.401215549885


In [60]:
res_dict

{'em': 74.24607799008619,
 'pso': 74.48534941906306,
 'ref_em_score': 74.16039295495584}

In [12]:
import numpy as np
from utils import _givens_rotation_matrix_entries

def QRGivens_slow(A):
    if np.linalg.det(A) < 0:
        A = -A
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)
    phi_list = []

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
    i = 0
    for (row, col) in zip(rows, cols):
        i += 1

        # Compute Givens rotation matrix and
        # zero-out lower triangular matrix entries.
        (c, s) = _givens_rotation_matrix_entries(R[col, col], R[row, col])


        phi = np.arccos(c)
        # if sin(phi) < 0
        if s > 0:
            phi = -phi

        # Turning first element into 1 instead of -1
        if c * R[col, col] - s * R[row, col] < 0:
            phi = phi - np.pi
            c = -c
            s = -s

        G = np.identity(num_rows)
        G[col, col] = c
        G[row, row] = c
        phi_list.append(phi)
        G[row, col] = s
        G[col, row] = -s

        R = np.dot(G, R)

        Q = np.dot(Q, G.T)

    return Q, R, phi_list


def QRGivens_fast(A):
    if np.linalg.det(A) < 0:
        A = -A
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)
    phi_list = []

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
    i = 0
    for (row, col) in zip(rows, cols):
        i += 1

        # Compute Givens rotation matrix and
        # zero-out lower triangular matrix entries.
        (c, s) = _givens_rotation_matrix_entries(R[col, col], R[row, col])


        phi = np.arccos(c)
        # if sin(phi) < 0
        if s > 0:
            phi = -phi

        # Turning first element into 1 instead of -1
        if c * R[col, col] - s * R[row, col] < 0:
            phi = phi - np.pi
            c = -c
            s = -s

        G = np.identity(num_rows)
        G[col, col] = c
        G[row, row] = c
        phi_list.append(phi)
        G[row, col] = s
        G[col, row] = -s

        R[col], R[row] = R[col]*c + R[row]*(-s), R[col]*s + R[row]*c
        Q[:, col], Q[:, row] = Q[:, col]*c + Q[:, row]*(-s), Q[:, col]*s + Q[:, row]*c

        # R = np.dot(G, R)

        # Q = np.dot(Q, G.T)

    return Q, R, phi_list


In [14]:
from utils import QRGivens

dim = 50

from sklearn.datasets import make_spd_matrix
A = make_spd_matrix(dim)

%timeit QRGivens_fast(A)
%timeit np.linalg.qr(A)
%timeit np.linalg.eigh(A)
%timeit QRGivens_slow(A)

25.8 ms ± 85.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
67.1 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
361 µs ± 40.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
32.4 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:

%timeit np.linalg.eigh(A)

17.2 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
